In [84]:
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

import numpy as np

# Clasificación

In [85]:
iris = load_iris()

In [86]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [87]:
iris.feature_names, iris.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [88]:
X=iris.data[:,2:]
y=iris.target

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [90]:
clf = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)

In [91]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)

In [92]:
cross_val_score(clf, X_train, y_train, cv=10).mean()

0.9553030303030303

In [68]:
clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [74]:
iris.target_names[clf.predict([[5, 1.5]])[0]]

'versicolor'

In [78]:
clf.max_features_,clf.n_classes_,clf.n_features_,clf.n_outputs_

(2, 3, 2, 1)

In [80]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

In [81]:
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth

while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

The binary tree structure has 5 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 1] <= 0.800000011920929 else to node 2.
	node=1 leaf node.
	node=2 test node: go to node 3 if X[:, 1] <= 1.75 else to node 4.
		node=3 leaf node.
		node=4 leaf node.



In [93]:
# First let's retrieve the decision path of each sample. The decision_path
# method allows to retrieve the node indicator functions. A non zero element of
# indicator matrix at the position (i, j) indicates that the sample i goes
# through the node j.

node_indicator = clf.decision_path(X_test)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = clf.apply(X_test)

# Now, it's possible to get the tests that were used to predict a sample or
# a group of samples. First, let's make it for the sample.

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

Rules used to predict sample 0: 
decision id node 0 : (X_test[0, 1] (= 2.4) > 0.800000011920929)
decision id node 2 : (X_test[0, 1] (= 2.4) > 1.75)

The following samples [0, 1] share the node [0 2] in the tree
It is 40.0 % of all nodes.


# Regresión

In [94]:
from sklearn.tree import DecisionTreeRegressor

In [95]:
reg = DecisionTreeRegressor(max_leaf_nodes=3, random_state=0)

In [96]:
reg.fit(X_train, y_train)

DecisionTreeRegressor(max_leaf_nodes=3, random_state=0)

In [98]:
cross_val_score(reg, X_train, y_train, cv=10).mean()

0.9122295246127294

In [99]:
reg.predict([[5, 1.5]])

array([1.97435897])